In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')


def naive_nodewise_regression(Y_star, lambda_grid=None):
    """
    Implements Naive Nodewise Regression (Section 5.1.2).
    Uses GIC (Generalized Information Criterion) as in the paper.
    
    Parameters:
    -----------
    Y_star : np.ndarray, shape (n, p)
        Demeaned returns matrix (time x assets)
    lambda_grid : list or None
        Grid of lambda values to try. If None, creates default grid.
    
    Returns:
    --------
    Theta_hat : np.ndarray, shape (p, p)
        Estimated precision matrix
    """
    n, p = Y_star.shape

    # Initialize matrices
    Theta_hat = np.zeros((p, p))
    tau_squared = np.zeros(p)
    
    # Create lambda grid if not provided
    if lambda_grid is None:
        lambda_grid = np.logspace(-3, 1, 50)
    
    # For each asset j
    for j in range(p):
        # Step 1: Get y_j (target) and Y_{-j} (predictors)
        y_j = Y_star[:, j]
        Y_minus_j = np.delete(Y_star, j, axis=1)
        
        # Step 2-3: Estimate gamma_j using Lasso with GIC
        best_gic = np.inf
        best_lambda = lambda_grid[0]
        best_gamma = None
        best_ssr = None
        
        for lam in lambda_grid:
            lasso = Lasso(alpha=2*lam, fit_intercept=False, max_iter=10000)
            lasso.fit(Y_minus_j, y_j)
            gamma_j = lasso.coef_
            
            # Compute SSR and number of non-zero coefficients
            residuals = y_j - Y_minus_j @ gamma_j
            ssr = np.sum(residuals ** 2)
            sigma_sq_lambda = ssr / n
            q_lambda = np.sum(np.abs(gamma_j) > 1e-8)
            
            # Compute GIC
            if sigma_sq_lambda > 1e-10:
                gic = np.log(sigma_sq_lambda) + q_lambda * (np.log(p) / n) * np.log(np.log(n))
            else:
                gic = np.inf
            
            if gic < best_gic:
                best_gic = gic
                best_lambda = lam
                best_gamma = gamma_j.copy()
                best_ssr = ssr
        
        gamma_j_star = best_gamma
        tau_squared[j] = best_ssr / n + best_lambda * np.sum(np.abs(gamma_j_star))

        # Step 5: Form the j-th row of Theta_hat
        Theta_hat[j, j] = 1 / tau_squared[j]
        off_diag = -gamma_j_star / tau_squared[j]
        Theta_hat[j, :j] = off_diag[:j]
        Theta_hat[j, j+1:] = off_diag[j:]
    
    # Step 6: Symmetrize
    Theta_hat_sym = (Theta_hat + Theta_hat.T) / 2
    
    return Theta_hat_sym


def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights (Section 6.1).
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # w* = (Θ 1_p) / (1_p' Θ 1_p)
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        # Fallback to equal weights if precision matrix is near-singular
        return ones_p / p
    
    w_star = numerator / denominator
    
    return w_star


def load_finbert_signals(signals_path):
    """
    Load FinBERT monthly signals from CSV file.
    
    Parameters:
    -----------
    signals_path : str
        Path to monthly_signals.csv file
    
    Returns:
    --------
    signals_df : pd.DataFrame
        DataFrame with columns: symbol, company, year_month, signal, avg_sentiment_score
    """
    try:
        signals_df = pd.read_csv(signals_path)
        # Convert year_month to datetime (end of month)
        signals_df['date'] = pd.to_datetime(signals_df['year_month']) + pd.offsets.MonthEnd(0)
        return signals_df
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load FinBERT signals: {e}")
        return pd.DataFrame(columns=['symbol', 'company', 'year_month', 'signal', 'date'])


def get_buy_signal_permnos_for_date(signals_df, ticker_to_permno, date):
    """
    Get set of permnos with 'buy' signals for a specific date.
    
    Parameters:
    -----------
    signals_df : pd.DataFrame
        FinBERT signals dataframe
    ticker_to_permno : dict
        Mapping from ticker symbol to permno
    date : pd.Timestamp
        Date to get signals for
    
    Returns:
    --------
    permno_set : set
        Set of permnos with buy signals on this date
    """
    # Get signals for this date
    date_signals = signals_df[signals_df['date'] == date]
    
    # Filter for buy signals only
    buy_signals = date_signals[date_signals['signal'] == 'buy']
    
    # Convert tickers to permnos
    permnos = set()
    for ticker in buy_signals['symbol'].values:
        if ticker in ticker_to_permno:
            permnos.add(ticker_to_permno[ticker])
    
    return permnos


def create_ticker_to_permno_mapping(df):
    """
    Create a mapping from ticker to permno from the returns dataframe.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Returns dataframe with 'ticker' and 'permno' columns
    
    Returns:
    --------
    ticker_to_permno : dict
        Mapping from ticker to permno (uses most recent permno for each ticker)
    """
    if 'ticker' not in df.columns:
        raise ValueError("DataFrame must have 'ticker' column for mapping")
    
    # Drop NaN tickers
    valid_df = df[df['ticker'].notna()].copy()
    
    # Get the most recent permno for each ticker
    ticker_to_permno = valid_df.groupby('ticker')['permno'].last().to_dict()
    
    return ticker_to_permno


def backtest_nodewise_gmv_finbert(df, 
                                   signals_path='monthly_signals.csv',
                                   test_start_date='2020-01-31', 
                                   test_end_date='2024-11-30',
                                   lookback_window=180,
                                   transaction_cost=0.001,
                                   verbose=True):
    """
    Backtest Nodewise + GMV strategy using FinBERT buy signals.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ticker, ret_fwd_1
    signals_path : str
        Path to monthly_signals.csv file
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.001 = 10 bps)
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results_df : pd.DataFrame
        DataFrame with columns: date, portfolio_return, cumulative_return
    metrics : dict
        Overall performance metrics
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Create ticker to permno mapping
    if verbose:
        print("Creating ticker to permno mapping...")
    ticker_to_permno = create_ticker_to_permno_mapping(df)
    if verbose:
        print(f"Mapped {len(ticker_to_permno)} unique tickers to permnos")
    
    # Load FinBERT signals
    if verbose:
        print(f"Loading FinBERT signals from {signals_path}...")
    signals_df = load_finbert_signals(signals_path)
    if len(signals_df) == 0:
        raise ValueError("No FinBERT signals loaded")
    
    if verbose:
        print(f"Loaded {len(signals_df)} monthly signals")
        print(f"Signal distribution:")
        print(signals_df['signal'].value_counts())
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results
    portfolio_returns = []
    portfolio_dates = []
    portfolio_weights_list = []
    portfolio_turnover_list = []
    portfolio_gross_returns = []
    
    # Track weights by permno
    prev_weights_dict = {}
    prev_oos_returns_dict = {}
    prev_gross_return = 0.0
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print("STARTING BACKTEST WITH FINBERT BUY SIGNALS")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        
        # Get buy signal permnos for current date
        allowed_permnos = get_buy_signal_permnos_for_date(
            signals_df, ticker_to_permno, current_date
        )
        
        if len(allowed_permnos) == 0:
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ No buy signals, skipping period")
            continue
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window, filtered by allowed permnos
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date) &
                        (df['permno'].isin(allowed_permnos))]
        
        # Pivot to get returns matrix (time x assets)
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        # Filter assets with any NaNs in this window
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f"  Buy signals: {len(allowed_permnos)} | Assets w/ data: {p_current}")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), using prev weights")
            # Filter previous weights to only allowed permnos
            new_weights_dict = {k: v for k, v in prev_weights_dict.items() if k in allowed_permnos}
        else:
            try:
                # Demean the returns
                Y_bar = Y.mean(axis=0)
                Y_star = Y - Y_bar
                
                if verbose:
                    print(f"  Running Nodewise Regression...")
                Theta_hat = naive_nodewise_regression(Y_star)
                
                if verbose:
                    print(f"  Computing GMV weights...")
                w_star = gmv_weights(Theta_hat)
                
                # Create weights dictionary
                new_weights_dict = {asset: w_star[i] for i, asset in enumerate(current_assets)}
                
            except Exception as e:
                if verbose:
                    print(f"  ✗ Error: {e}")
                    print(f"  Using previous weights")
                # Filter previous weights to only allowed permnos
                new_weights_dict = {k: v for k, v in prev_weights_dict.items() if k in allowed_permnos}

        # Normalize weights to sum to 1
        weight_sum = sum(new_weights_dict.values())
        if weight_sum > 1e-10:
            new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum, using previous weights")
            new_weights_dict = {k: v for k, v in prev_weights_dict.items() if k in allowed_permnos}
            weight_sum = sum(new_weights_dict.values())
            if weight_sum > 1e-10:
                new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        
        # --- 3. OOS Returns & Transaction Costs ---
        
        # Get out-of-sample returns for current month (only for allowed permnos)
        oos_data = df[(df['datadate'] == current_date) & (df['permno'].isin(allowed_permnos))]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        
        # Filter out NaN returns
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Find common assets between weights and returns
        common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
        
        if len(common_assets) == 0:
            if verbose:
                print("  ⚠ No common assets with valid returns, skipping period")
            continue
        
        # Filter to common assets and renormalize
        common_weights = {a: new_weights_dict[a] for a in common_assets}
        common_weight_sum = sum(common_weights.values())
        if common_weight_sum > 1e-10:
            common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum after filtering, skipping period")
            continue
        
        # Compute gross portfolio return
        gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
        
        # Sanity check
        if np.isnan(gross_return) or np.isinf(gross_return):
            if verbose:
                print(f"  ⚠ Invalid gross return: {gross_return}, skipping period")
            continue
        
        # Calculate transaction costs
        if len(prev_weights_dict) > 0:
            all_traded_assets = set(common_weights.keys()) | set(prev_weights_dict.keys())
            
            # Adjust previous weights
            adjusted_prev = {}
            for asset in all_traded_assets:
                prev_w = prev_weights_dict.get(asset, 0.0)
                
                if asset in prev_oos_returns_dict:
                    prev_r = prev_oos_returns_dict[asset]
                    if prev_gross_return > -0.99:
                        adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
                else:
                    adjusted_prev[asset] = 0.0
            
            # Renormalize adjusted weights
            adj_sum = sum(adjusted_prev.get(a, 0.0) for a in common_weights.keys())
            if adj_sum > 1e-10:
                adjusted_prev_normalized = {k: adjusted_prev.get(k, 0.0)/adj_sum 
                                           for k in common_weights.keys()}
            else:
                adjusted_prev_normalized = {k: 0.0 for k in common_weights.keys()}
            
            # Turnover
            turnover = sum(abs(common_weights.get(a, 0.0) - adjusted_prev_normalized.get(a, 0.0)) 
                          for a in all_traded_assets)
            
            # Transaction cost
            tc = transaction_cost * (1 + gross_return) * turnover
        else:
            # First period
            turnover = sum(abs(w) for w in common_weights.values())
            tc = transaction_cost * turnover
        
        # Net return
        net_return = gross_return - tc
        
        # Store results
        portfolio_returns.append(net_return)
        portfolio_dates.append(current_date)
        portfolio_weights_list.append(common_weights.copy())
        portfolio_turnover_list.append(turnover)
        portfolio_gross_returns.append(gross_return)
        
        # Update previous values for next iteration
        prev_weights_dict = common_weights.copy()
        prev_oos_returns_dict = {a: oos_returns_dict[a] for a in common_assets}
        prev_gross_return = gross_return
        
        if verbose:
            print(f"  Gross: {gross_return:>8.5f} | Turnover: {turnover:>6.4f} | "
                  f"TC: {tc:>8.6f} | Net: {net_return:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results ---
    results_df = pd.DataFrame({
        'date': portfolio_dates,
        'portfolio_return': portfolio_returns,
        'portfolio_gross_return': portfolio_gross_returns,
        'portfolio_weights': portfolio_weights_list,
        'portfolio_turnover': portfolio_turnover_list
    })
    results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
    
    # Compute overall metrics
    if len(portfolio_returns) > 0:
        mean_return = np.mean(portfolio_returns)
        variance = np.var(portfolio_returns, ddof=1)
        sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
        
        # Annualized metrics (monthly data)
        annual_return = mean_return * 12
        annual_volatility = np.sqrt(variance * 12)
        annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
        
        metrics = {
            'mean_return': mean_return,
            'variance': variance,
            'sharpe_ratio': sharpe_ratio,
            'annual_return': annual_return,
            'annual_volatility': annual_volatility,
            'annual_sharpe_ratio': annual_sharpe,
            'total_return': results_df['cumulative_return'].iloc[-1],
            'avg_turnover': np.mean(portfolio_turnover_list),
            'n_periods': len(portfolio_returns)
        }
    else:
        metrics = {
            'mean_return': 0,
            'variance': 0,
            'sharpe_ratio': 0,
            'annual_return': 0,
            'annual_volatility': 0,
            'annual_sharpe_ratio': 0,
            'total_return': 0,
            'avg_turnover': 0,
            'n_periods': 0
        }
    
    return results_df, metrics

In [3]:
df = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = df.groupby('permno')['ret_excess'].shift(-1)

In [8]:
results_df, metrics = backtest_nodewise_gmv_finbert(
    df=df,
    signals_path='monthly_signals.csv',
    test_start_date='2020-01-31',
    test_end_date='2024-11-30',
    lookback_window=180,
    transaction_cost=0.001,
    verbose=True
)

print("\nPerformance Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value}")

Creating ticker to permno mapping...
Mapped 1664 unique tickers to permnos
Loading FinBERT signals from monthly_signals.csv...
Loaded 1806 monthly signals
Signal distribution:
signal
sell    799
buy     653
hold    354
Name: count, dtype: int64
STARTING BACKTEST WITH FINBERT BUY SIGNALS

[1/59] Date: 2020-01-31
  Window: 2005-01-31 to 2019-12-31
  Buy signals: 7 | Assets w/ data: 5
  Running Nodewise Regression...
  Computing GMV weights...
  Gross: -0.13202 | Turnover: 1.0000 | TC: 0.000000 | Net: -0.13202

[2/59] Date: 2020-02-29
  Window: 2005-02-28 to 2020-01-31
  Buy signals: 5 | Assets w/ data: 3
  Running Nodewise Regression...
  Computing GMV weights...
  Gross: -0.07091 | Turnover: 0.9600 | TC: 0.000000 | Net: -0.07091

[3/59] Date: 2020-03-31
  Window: 2005-03-31 to 2020-02-29
  Buy signals: 5 | Assets w/ data: 2
  Running Nodewise Regression...
  Computing GMV weights...
  Gross:  0.05533 | Turnover: 1.0000 | TC: 0.000000 | Net:  0.05533

[4/59] Date: 2020-04-30
  Window: 20

In [10]:
results_df['portfolio_weights']

0     {54148: 0.06353613521591844, 23819: 0.07179758...
1     {52090: 0.5199768423883128, 10516: 0.185415790...
2     {16600: 0.6213324246202543, 56274: 0.378667575...
3     {82651: 0.21889998797634508, 86356: 0.12040648...
4     {46578: 0.35592928494166126, 52090: 0.32199116...
5     {46578: 0.5558684147396795, 77274: 0.226354802...
6     {77274: 0.3243599232801598, 84373: 0.396881631...
7     {46578: 0.5265825877318946, 49154: 0.205580092...
8     {87137: 0.015647930256273854, 88661: 0.0365193...
9     {21186: 0.10514897352831676, 23819: 0.03840542...
10    {19393: 0.13821660022700588, 17005: 0.14269574...
11    {60442: 0.5918406709320633, 52708: 0.197410768...
12    {35044: 0.07500460072647971, 15720: 0.31514302...
13    {15720: 0.26891582893081234, 57904: 0.09463949...
14    {21186: 0.08482901712043692, 49154: 0.14388740...
15    {41355: 0.049701389511887795, 11308: 0.3051606...
16    {16600: 0.5366823602003937, 87137: 0.041768132...
17    {52038: 0.17018974186765423, 11308: 0.3151